<a href="https://colab.research.google.com/github/rhks13/Vision_practice/blob/main/%EB%B2%A0%EC%9D%B4%EC%8A%A4%EB%9D%BC%EC%9D%B8_efficientnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 출처 : https://dacon.io/competitions/official/236082/codeshare/7887?page=1&dtype=recent

# **초기 설정**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)                              # random 모듈의 랜덤시드 고정
    os.environ['PYTHONHASHSEED'] = str(seed)       # os 모듈을 이용해 파이썬 해시 함수의 시드값 고정
    np.random.seed(seed)                           # numpy 모듈의 랜덤시드를 고정
    torch.manual_seed(seed)                        # torch 모듈의 랜덤시드 고정
    torch.cuda.manual_seed(seed)                   # torch cuda 모듈의 랜덤시드 고정
    torch.backends.cudnn.deterministic = True      # cuda gpu 사용하는 경우 랜덤시드 고정
    torch.backends.cudnn.benchmark = True          # 최적화를 위한 백엔드 사용

seed_everything(CFG['SEED']) # Seed 고정

# **데이터 전처리**

In [ ]:
# 이미지 불러오기
all_img_list = glob.glob('/content/drive/MyDrive/캐글&데이콘/데이콘 - 도배하자/data/train/*/*')

In [ ]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])
df

,img_path,label
0,/content/drive/MyDrive/캐글&데이콘/데이콘 -...,면불량
1,/content/drive/MyDrive/캐글&데이콘/데이콘 -...,면불량
2,/content/drive/MyDrive/캐글&데이콘/데이콘 -...,면불량
3,/content/drive/MyDrive/캐글&데이콘/데이콘 -...,면불량
4,/content/drive/MyDrive/캐글&데이콘/데이콘 -...,면불량
...,...,...
3452,/content/drive/MyDrive/캐글&데이콘/데이콘 -...,피스
3453,/content/drive/MyDrive/캐글&데이콘/데이콘 -...,피스
3454,/content/drive/MyDrive/캐글&데이콘/데이콘 -...,피스
3455,/content/drive/MyDrive/캐글&데이콘/데이콘 -...,피스


In [ ]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

In [ ]:
# 라벨 인코딩
le = preprocessing.LabelEncoder()      # LabelEncoder() -> 문자열 or 숫자형 데이터 => 숫자형으로 변환
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [ ]:
train['label']

2318    18
1085    18
1317    18
1595    18
1301    18
        ..
1941    18
2754    10
81       6
1763    18
908      7
Name: label, Length: 2419, dtype: int64

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms          # 이미지 변형을 위해 transforms 객체를 받아옴
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
#albumentations 라이브러리 -> 이미지 데이터 전처리 수행하는 코드
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']), # 이미지를 지정된 크기로 변환
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), # 이미지 정규화 -> 지정된 값은 imagenet 데이터셋을 사용해 미리 계산된 값
                                        max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2() #pytorch tensor 형식으로 변환-> CHW 형식(채널, 높이, 너비)
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
# Data Loader 생성

train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# **모델링**

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True) #사전학습된 에피션트 넷
        self.classifier = nn.Linear(1000, num_classes) # num_classes -> 모델이 분류해야 하는 클래스의 개수
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)                                  # 모델을 GPU 또는 CPU로 보내요
    criterion = nn.CrossEntropyLoss().to(device)      # 비용함수 : cross entropy, 얘도 GPU or CPU로 보낸다
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)): # train loader를 이용해서 이미지와 레이블을 가져옴
            imgs = imgs.float().to(device)            # img, lables 를 gpu 혹은 cpu로 보내
            labels = labels.to(device)
            
            optimizer.zero_grad()                     # optimizer의 gradient를 0으로 초기화
            
            output = model(imgs)                      # 모델을 통과시켜 output을 계산
            loss = criterion(output, labels)          # criterion 을 이용하여 output과 labels의 loss 계산
            
            loss.backward()                           # gradient 계산
            optimizer.step()                          # optimizer 업데이트
            
            train_loss.append(loss.item())            # train_loss 리스트에 loss 값 추가
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
         # 최종적으로 학습결과를 출력 
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:          # 만약 scheduler가 있으면 _var_score 이용해서 learning rate 조정
            scheduler.step(_val_score)
            
        if best_score < _val_score:        # 최상의 score와 model 뽑기
            best_score = _val_score
            best_model = model
    
    return best_model                      # 최종 모델 반환

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()                                                       # 모델을 evaluation mode로 전환 -> dropout, batch normalization 연산 적용 X
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():                                              # gradient 계산 비활성화
        for imgs, labels in tqdm(iter(val_loader)):                    # 각 배치 데이터에 대해 순전파 진행
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)                              # pred 와 true label 비교
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())                                # validation loss 계산
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')   # f1 score 계산
    
    return _val_loss, _val_score                                        # 최종 loss, score 반환

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
# ReduceLROnPlateau 스케줄러 -> validation loss를 지속적으로 모니터링 => loss 개선이 멈출 경우 학습률을 조정해 모델 학습 최적화
# patience: 개선 없이 몇번의 에포크를 지정할지
# threshold_mode : 개선 여부를 결정하는 방식
# min_lr, verbose : 최소 학습률, 로깅 설정
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 241MB/s]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.44367] Val Loss : [0.92651] Val Weighted F1 Score : [0.66904]


  0%|          | 0/76 [00:00<?, ?it/s]

...

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.05184] Val Loss : [1.21938] Val Weighted F1 Score : [0.74731]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.05808] Val Loss : [1.17381] Val Weighted F1 Score : [0.77229]


# **인퍼런스 & 제출**

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/캐글&데이콘/데이콘 - 도배하자/data/test.csv')

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

In [ ]:
#제출
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = preds
submit.to_csv('./baseline_submit.csv', index=False)